#### import libraries

In [3]:
import pandas as pd

## Data cleaning for Rosenberg results

In [121]:
rosenberg = pd.read_csv("../data/02_RSE/data_raw_tabulated.csv")
rosenberg.head()
# all types correct
# data looks clean and ready to use
# no NaNs, txt file in folder describes all variables and meaning of results


,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,gender,age,source,country
0,3,3,1,4,3,4,3,2,3,3,1,40,1,US
1,4,4,1,3,1,3,3,2,3,2,1,36,1,US
2,2,3,2,3,3,3,2,3,3,3,2,22,1,US
3,4,3,2,3,2,3,2,3,3,3,1,31,1,US
4,4,4,1,4,1,4,4,1,1,1,1,30,1,EU


### Adding all scores, and reshaping the table to fit DB model

In [122]:
q_lst = ["Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10"]

# adding all quesiton results
# scale is  1 to 4 with 1 being the min and 4 the max
# the results should therefore go from 0 to 40
rosenberg["overall_score"] = rosenberg[q_lst].sum(axis=1)
rosenberg = rosenberg.drop(q_lst, axis=1)


In [123]:
rosenberg.head()

,gender,age,source,country,overall_score
0,1,40,1,US,29
1,1,36,1,US,26
2,2,22,1,US,27
3,1,31,1,US,28
4,1,30,1,EU,25


In [124]:
# information on source is redundant 
rosenberg = rosenberg.drop("source", axis=1)
rosenberg.head()

,gender,age,country,overall_score
0,1,40,US,29
1,1,36,US,26
2,2,22,US,27
3,1,31,US,28
4,1,30,EU,25


In [125]:
# age index will be changed to F, M for better uderstanding
# M=1, F=2, 0=3 (for other and no answer)

rosenberg["gender"] = rosenberg["gender"].replace([1], "M").replace([2], "F").replace([3], "O")
rosenberg.head()

,gender,age,country,overall_score
0,M,40,US,29
1,M,36,US,26
2,F,22,US,27
3,M,31,US,28
4,M,30,EU,25


In [126]:
# ordering of columns to match DB
rosenberg = rosenberg[["country", "age", "overall_score", "gender"]]

### Value check and cleaning

In [127]:
# checked for 0s in age - decided to leave them in as age in not as important as the score
rosenberg.loc[rosenberg["age"]==0]
# check if age over 90 (assuming no one over 85 is taking these types of test online)
rosenberg = rosenberg[rosenberg["age"]<85]

# checked for 0s in all the scores (ie if total less than 10) overall score - deleted as its all about score and not that many present
rosenberg.loc[rosenberg["overall_score"]<10]
rosenberg = rosenberg[rosenberg["overall_score"]>=10]

In [128]:
# check if any left with:
# rosenberg.loc[rosenberg["overall_score"]<10]


In [129]:
rosenberg.describe()
# data looks good! :)

,age,overall_score
count,47856.000000,47856.000000
mean,26.263039,26.725573
std,12.596816,2.777309
min,0.000000,10.000000
25%,18.000000,25.000000
50%,22.000000,27.000000
75%,32.000000,28.000000
max,84.000000,40.000000


In [130]:
counts =  rosenberg.groupby("country").count()
# 175 countries - accurate as found from IP adresses usign Maxmind geoIP
# all countries with a count under 18 (median value) will be eliminated as sample too small to be representative
counts.describe()
# A1 and A2 are removed as they represent "Anonymous Proxy" and "Satellite Provider"
# following ISO 3166 Country Codes

counts.loc[counts["age"]<18].index
country_lst = ['A1', 'A2', 'AF', 'AG', 'AI', 'AM', 'AN', 'AP', 'AS', 'AW', 'AZ', 'BH', 'BM', 'BN',
       'BO', 'BQ', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CI', 'DO', 'DZ', 'EC', 'FJ',
       'FK', 'FO', 'FX', 'GD', 'GG', 'GI', 'GP', 'GT', 'GU', 'GY', 'HN', 'HT',
       'IM', 'KH', 'KN', 'KW', 'KY', 'KZ', 'LC', 'LR', 'LU', 'LY', 'MC', 'MD',
       'ME', 'ML', 'MM', 'MN', 'MO', 'MP', 'MU', 'MV', 'MZ', 'NC', 'NI', 'NP',
       'OM', 'PA', 'PG', 'PS', 'PY', 'RE', 'RW', 'SB', 'SC', 'SD', 'SL', 'SN',
       'SR', 'SV', 'SY', 'SZ', 'TL', 'TN', 'TZ', 'UG', 'UY', 'UZ', 'VC', 'VI',
       'YE', 'ZM', 'ZW']

rosenberg["Drop"] = rosenberg["country"].isin(country_lst)
rosenberg = rosenberg[rosenberg["Drop"]==False]

In [131]:
rosenberg = rosenberg.drop(["Drop"], axis=1)

In [132]:
#check through:
#rosenberg[rosenberg[counts["age"]>18]]
#check for correct columns and changes
rosenberg.head()

,country,age,overall_score,gender
0,US,40,29,M
1,US,36,26,M
2,US,22,27,F
3,US,31,28,M
4,EU,30,25,M


In [133]:
rosenberg.describe()
# data looks good:)

,age,overall_score
count,47395.000000,47395.000000
mean,26.256208,26.724148
std,12.605567,2.773928
min,0.000000,10.000000
25%,18.000000,25.000000
50%,22.000000,27.000000
75%,32.000000,28.000000
max,84.000000,40.000000


###  ** needs more work for country code

In [134]:
# list of countries in the DB 
countries_r = rosenberg.groupby("country").count().reset_index()
country_lst_r = countries_r["country"].tolist()

In [135]:
rosenberg["country_id"] = rosenberg["country"]
rosenberg = rosenberg.replace({"country_id":country_code_dict})
rosenberg.head()
# as DB was based on the ISO naming, all values should be correct, no non numeric values

,country,age,overall_score,gender,country_id
0,US,40,29,M,234
1,US,36,26,M,234
2,US,22,27,F,234
3,US,31,28,M,234
4,EU,30,25,M,70


#### saving files

In [136]:
rosenberg.to_csv("../data/01_clean_data/rosenberg_table.csv") 

## Data cleaning for Hofstede report

In [20]:
hof = pd.read_excel("../data/03_society/ibm.xlsx")


### Data cleaning, reshaping to DB 

In [21]:
#change name for consistency
hof = hof.rename(columns={"Individualism_vs_collectivism":"individualism", "Countries":"country", "Power_distance":"power_distance"})

#select info watned
hof_columns = ["country", "power_distance", "individualism"]
hof = hof[hof_columns]

# all types correct
hof.head(100)

# data looks clean, a few NaNs, it sems where the power distance is a NaNs, the individualism is a NaN too
# all NaN rows will be removed. 
hof = hof.dropna().reset_index(drop=True)

hof.head()
hof.describe()
# data looks clean

,power_distance,individualism
count,69.000000,69.000000
mean,59.536232,42.913043
std,21.748279,24.326057
min,11.000000,2.000000
25%,40.000000,20.000000
50%,63.000000,39.000000
75%,76.000000,63.000000
max,104.000000,91.000000


#### ** needs more work for country code

In [22]:
# list of countries in the DB
countries_h = hof.groupby("country").count().reset_index()
country_lst_h = countries_h["country"].tolist()

In [23]:
# using dictionary defined below, replace values accordingly to country intexes
hof["country_id"] = hof["country"] 
hof = hof.replace({"country_id":country_dict})
hof.head()

,country,power_distance,individualism,country_id
0,Arabic countries,80.0,38.0,Arabic countries
1,Argentina,49.0,46.0,10
2,Australia,36.0,90.0,13
3,Austria,11.0,55.0,12
4,Bangladesh,80.0,20.0,19


In [24]:
#separate non-numeric values
hof["numeric"] = hof["country_id"].str.isnumeric()

#saved grouped values for regions separately - might be useful later
grouped_values_hof = hof[hof["numeric"]==False]
grouped_values_hof

#frop non numeric
hof = hof[hof["numeric"]!=False]
hof = hof.drop(["numeric"], axis=1).reset_index(drop=True)

####  save files

In [25]:
hof.to_csv("../data/01_clean_data/hofstede_report.csv") 
grouped_values_hof.to_csv("../data/01_clean_data/hofstede_report_grouped_regions.csv") 

## Cleaning ISO country list and making dictionaries 

In [9]:
ISO = pd.read_csv("../data/02_RSE/iso3166_countries.csv")
ISO.tail()

,country_code,country
249,YE,Yemen
250,YT,Mayotte
251,ZA,South Africa
252,ZM,Zambia
253,ZW,Zimbabwe


In [10]:
# remove top 3 rows as not needed
ISO = ISO.loc[3:].reset_index(drop=True)
ISO.head()
#data is now OK

,country_code,country
0,AD,Andorra
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Antigua and Barbuda
4,AI,Anguilla


In [11]:
# make into dictioanry to be able to easily replace 
lst_index = [i for i in range(251)]
lst_country_code = ISO["country_code"].to_list()
lst_country = ISO["country"].to_list()


In [12]:
country_code_dict = dict(zip(lst_country_code, lst_index))
country_dict = dict(zip(lst_country, lst_index))


#### saving files

In [13]:
ISO.to_csv("../data/01_clean_data/ISO_country_codes.csv") 